### Imports

In [8]:
import requests
import pandas as pd
import numpy as np

In [145]:
%cd ..

/Users/Marie/code/MarieMCodes/re_value_predictor


/Users/Marie/.pyenv/versions/3.10.6/envs/re_value_predictor/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [155]:
from project_code.data import load_csv, tidy_df

In [156]:
#%load_ext autoreload
#%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build and test API function

In [ ]:
# sample http request
#https://nominatim.openstreetmap.org/search.php?q=26+corporation+street+london&dedupe=0&limit=3&format=jsonv2

In [99]:
def get_lat(address):
    url="https://nominatim.openstreetmap.org/"
    params={"q": address, "format": "json"}
    response=requests.get(url,params=params).json()
    lat=response[0]["lat"]
    return lat

#url=f"https://nominatim.openstreetmap.org/search?q={address}&format=json"

In [93]:
def get_lon(address):
    url="https://nominatim.openstreetmap.org/"
    params={"q": address, "format": "json"}
    response=requests.get(url,params=params).json()
    lon=response[0]["lon"]
    return lon


In [35]:
test['street_name'][1]

'10 oxford street'

In [223]:
def get_coordinates(address):
    url="https://nominatim.openstreetmap.org/"
    try:
        params={"q": address, "format": "json"}
        response=requests.get(url,params=params).json()
        lat=response[0]["lat"]
        lon=response[0]["lon"]
        full_address=response[0]["display_name"]
        return lat, lon, full_address
    except:
        return "ERROR"
        

In [188]:
address='2 CORPORATION STREET, London, N7 9EE'
#get_lat(address) 
#get_lon(address)
get_coordinates(address)

('51.53216985',
 '0.01500748219379946',
 '2, Corporation Street, West Ham, London Borough of Newham, London, Greater London, England, E15 3DY, United Kingdom')

In [56]:
test=pd.DataFrame({'street_name':['2 corporation street', '10 oxford street', '3 ranston street']})
test['lat']=''
test['lon']=''

In [57]:
test

,street_name,lat,lon
0,2 corporation street,,
1,10 oxford street,,
2,3 ranston street,,


In [91]:
# Make API call and fill new column in df with results
test['lat']=test['street_name'].apply(get_lat)

In [158]:
test['lat_lon_a']=test['street_name'].apply(get_coordinates)

In [102]:
test['lon']=test['street_name'].apply(get_lon)
# now separate list out into 2 fields as float 
test

In [159]:
test['lat1']=test['lat_lon_a'].apply(lambda x: x[0])
test['lon2']=test['lat_lon_a'].apply(lambda x: x[1])
test['full_address_api']=test['lat_lon_a'].apply(lambda x: x[2])

In [160]:
test

,street_name,lat,lon,lat_lon,lat1,lon2,lat_lon_a,full_address_api
0,2 corporation street,53.5377638,-2.6335841399051487,"(53.5377638, -2.6335841399051487)",53.5377638,-2.6335841399051487,"(53.5377638, -2.6335841399051487, Philbin Offi...","Philbin Office, 2, Corporation Street, Wigan P..."
1,10 oxford street,-36.1572186,174.2378783,"(-36.1572186, 174.2378783)",-36.1572186,174.2378783,"(-36.1572186, 174.2378783, 10, Oxford Street, ...","10, Oxford Street, Whakapirau, Pahi, Kaipara D..."
2,3 ranston street,51.521691000000004,-0.1676722360616832,"(51.521691000000004, -0.1676722360616832)",51.521691000000004,-0.1676722360616832,"(51.521691000000004, -0.1676722360616832, 3, R...","3, Ranston Street, St. John's Wood, London, Gr..."


In [87]:
get_coordinate('2 Corporation Street London')

'51.53216985'

### Get DF and explore same street name - different postcode scenario

In [169]:
data=load_csv()

In [170]:
df=tidy_df(data)

In [163]:
!pwd

/Users/Marie/code/MarieMCodes/re_value_predictor


In [171]:
df.head()

,price,date,postcode,property_type,property_age,ownership,number,additional_info,street,locality,town,borough,county,year,month,day,full_property_number
3,405000,95-04-07 00:00,N1 2JU,S,N,F,49,,CANONBURY PARK NORTH,LONDON,LONDON,ISLINGTON,GREATER LONDON,1995,4,7,49
120,620000,95-09-01 00:00,SW15 2NU,T,N,F,77,,DEODAR ROAD,LONDON,LONDON,WANDSWORTH,GREATER LONDON,1995,9,1,77
132,200500,95-11-20 00:00,SE23 2QA,D,N,F,40,,SUNDERLAND ROAD,LONDON,LONDON,LEWISHAM,GREATER LONDON,1995,11,20,40
138,270000,95-09-01 00:00,SW5 0DU,F,N,L,COLEHERNE COURT,FLAT 188,REDCLIFFE GARDENS,LONDON,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,1995,9,1,COLEHERNE COURTFLAT 188
251,210000,95-05-19 00:00,SW19 7HD,T,N,F,21,,PINE GROVE,LONDON,LONDON,MERTON,GREATER LONDON,1995,5,19,21


In [176]:
street_data=df.groupby('street').count()['postcode'].sort_values(ascending=True)

In [179]:
street_data[street_data>1]

street
HALFORD CLOSE        2
TILLETT WAY          2
HALIFAX CLOSE        2
TIVERTON CLOSE       2
TODD CLOSE           2
                  ... 
THE AVENUE        3266
CHURCH ROAD       3342
PARK ROAD         3703
LONDON ROAD       3773
HIGH STREET       6179
Name: postcode, Length: 35486, dtype: int64

In [ ]:
# can take a sample and then compare postcode from api w postcode from address 
df.columns

# dropping unneccessary columns
df.drop(columns=['price', 'date', 'property_type', 'property_age',
       'ownership', 'locality', 'town', 'county', 'year', 'month', 'day'],  inplace=True)

In [222]:
# choosing remaining columns
cols=['borough','postcode','additional_info','full_property_number', 'number','street']
df[cols]

,borough,postcode,additional_info,full_property_number,number,street
3,ISLINGTON,N1 2JU,,49,49,CANONBURY PARK NORTH
120,WANDSWORTH,SW15 2NU,,77,77,DEODAR ROAD
132,LEWISHAM,SE23 2QA,,40,40,SUNDERLAND ROAD
138,KENSINGTON AND CHELSEA,SW5 0DU,FLAT 188,COLEHERNE COURTFLAT 188,COLEHERNE COURT,REDCLIFFE GARDENS
251,MERTON,SW19 7HD,,21,21,PINE GROVE
...,...,...,...,...,...,...
3623314,HAMMERSMITH AND FULHAM,W12 7GR,APARTMENT 106,5APARTMENT 106,5,WOOD CRESCENT
3623315,WALTHAM FOREST,E4 9QB,6,BAILEY COURT6,BAILEY COURT,CASTLE AVENUE
3623316,BARKING AND DAGENHAM,RM6 6SX,,12,12,HENLEY GARDENS
3623317,WALTHAM FOREST,E4 9SY,,108B,108B,NEW ROAD


In [215]:
# creating new column to use for api call 
df['address']=df['number']+ ' ' + df['street'] + " London " + df['postcode']
tmp=df.drop(columns=['number', 'additional_info', 'full_property_number'])
tmp.head()

,postcode,street,borough,address
3,N1 2JU,CANONBURY PARK NORTH,ISLINGTON,49 CANONBURY PARK NORTH London N1 2JU
120,SW15 2NU,DEODAR ROAD,WANDSWORTH,77 DEODAR ROAD London SW15 2NU
132,SE23 2QA,SUNDERLAND ROAD,LEWISHAM,40 SUNDERLAND ROAD London SE23 2QA
138,SW5 0DU,REDCLIFFE GARDENS,KENSINGTON AND CHELSEA,COLEHERNE COURT REDCLIFFE GARDENS London SW5 0DU
251,SW19 7HD,PINE GROVE,MERTON,21 PINE GROVE London SW19 7HD


### Making API call on sample data

In [216]:
df_sample=tmp.sample(10000,random_state=42)
df_sample

,postcode,street,borough,address
1154282,W1K 1HA,SOUTH AUDLEY STREET,CITY OF WESTMINSTER,CHESTERFIELD HOUSE SOUTH AUDLEY STREET London ...
1519721,NW11 6BS,BEAUFORT PARK,BARNET,30 BEAUFORT PARK London NW11 6BS
1675228,NW3 5TN,NETHERHALL GARDENS,CAMDEN,"NETHERHALL MANSIONS, 30 NETHERHALL GARDENS Lon..."
3101266,SW14 7PF,PENRHYN CRESCENT,RICHMOND UPON THAMES,3 PENRHYN CRESCENT London SW14 7PF
1565916,SW13 0NS,THE TERRACE,RICHMOND UPON THAMES,ELM BANK MANSIONS THE TERRACE London SW13 0NS
...,...,...,...,...
2991254,HA9 8PJ,AYLANDS CLOSE,BRENT,28 AYLANDS CLOSE London HA9 8PJ
3249935,EN2 8FJ,ENDERS CLOSE,ENFIELD,3 ENDERS CLOSE London EN2 8FJ
3020112,SE13 7EG,ELIOT PARK,LEWISHAM,21 ELIOT PARK London SE13 7EG
2936742,RM2 6GH,KIDMAN CLOSE,HAVERING,TALLIS COURT KIDMAN CLOSE London RM2 6GH


In [206]:
df_sample.dtypes

postcode    object
street      object
borough     object
address     object
dtype: object

In [219]:
# because it's a sample, first row equals sample number... 
df_sample['postcode'][1154282]

'W1K 1HA'

In [211]:
get_coordinates('30 BEAUFORT PARK, London, NW11 6BS')


('51.58911845',
 '-0.19421753291426574',
 'Beaufort Park, Temple Fortune, London Borough of Barnet, London, Greater London, England, United Kingdom')

In [224]:
get_coordinates('21 ELIOT PARK London')

('51.46722055',
 '-0.00798084600200604',
 '21, Eliot Park, Baizdon Road Estate, Deptford, London Borough of Lewisham, London, Greater London, England, SE13 7PJ, United Kingdom')

In [ ]:
df_sample['lat_lon_a']=df_sample['address'].apply(get_coordinates)
df_sample['lat']=df_sample['lat_lon_a'].apply(lambda x: x[0])
df_sample['lon']=df_sample['lat_lon_a'].apply(lambda x: x[1])
df_sample['full_address_api']=df_sample['lat_lon_a'].apply(lambda x: x[2])
df_sample.head()

In [ ]:
#next, need to merge street name and full address and postcode into string
# better get address with one api call and fill both values in one column and separate after in df?  and not lat lon in 2 separate api calls. 